## When do I need to care about missing data?

If you don't know some of the values in your dataset, then you can't

- calculate summary statistics
- run most machine learning models

## Why is data sometimes missing?

There are many reasons!

- For survey data, perhaps someone declined to answer a question.
- For website data, perhaps some users installed privacy tools so you could not track their behavior.
- For sensor data, perhaps a sensor was not working, or a signal was too small to detect.

Understanding why data is missing is crucial to deciding how best to deal with it.

## How can I handle missing data?

There are essentially two choices:

- Delete the missing data (drop the rows from your dataset)
- Make up reasonable values (imputation)

## What are the steps for handling missing data?

1. Standardize how missing values are recorded
2. Quantify how much missing data you have
3. Either delete or impute the missing values
4. Run your model (or do other analysis)
5. Check the effect of missing values on your model

## What Python packages can I use for handling missing data?

- **pandas** (used here)
- **scikit-learn** (used here)
- **PyCaret**

## Limitations

- **Time series aren't covered here**. In time series data, rows at nearby time points are related, so they have their own methods for dealing with missing data.
- **Survival analysis isn't covered here**. If values are missing because they exceed a threshold, then you have a survival analysis problem, which requires different techniques.
- **Multiple imputation isn't covered here**. The most sophisticated techniques for dealing with missing data involve multiple imputation, but that isn't well supported in scikit-learn.
- **Imputation is not valid when missing values are "missing not at random"**. If there is a pattern to the missingness caused by variables that aren't in the dataset, then the techniques discussed here aren't valid. 

## Case study: Mammalian sleep durations

Let's explore a popular dataset on mammalian sleep durations, `msleep`. The dataset was cribbed from a 2007 paper by V. M. Savage and G. B. West and Wikipedia, and was popularized by R's **ggplot2** package. It's available in Python via the **plotnine** package.

Since the dataset is a dataframe, we'll import **pandas** too.

In [ ]:
import pandas as pd
from plotnine.data import msleep

Some extra code has been added to demonstrate issues with dirty data.

In [ ]:
msleep_dirty = msleep.copy()
msleep_dirty["conservation"] = msleep_dirty["conservation"].cat.set_categories(["lc", "nt", "vu", "en", "cr", "ew", "ex", "unknown"], ordered=True).fillna(value="unknown")
msleep_dirty["sleep_rem"] = msleep_dirty["sleep_rem"].fillna(value=-999)
msleep_dirty

,name,genus,vore,order,conservation,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,Cheetah,Acinonyx,carni,Carnivora,lc,12.1,-999.0,NaN,11.9,NaN,50.000
1,Owl monkey,Aotus,omni,Primates,unknown,17.0,1.8,NaN,7.0,0.01550,0.480
2,Mountain beaver,Aplodontia,herbi,Rodentia,nt,14.4,2.4,NaN,9.6,NaN,1.350
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc,14.9,2.3,0.133333,9.1,0.00029,0.019
4,Cow,Bos,herbi,Artiodactyla,unknown,4.0,0.7,0.666667,20.0,0.42300,600.000
...,...,...,...,...,...,...,...,...,...,...,...
78,Tree shrew,Tupaia,omni,Scandentia,unknown,8.9,2.6,0.233333,15.1,0.00250,0.104
79,Bottle-nosed dolphin,Tursiops,carni,Cetacea,unknown,5.2,-999.0,NaN,18.8,NaN,173.330
80,Genet,Genetta,carni,Carnivora,unknown,6.3,1.3,NaN,17.7,0.01750,2.000
81,Arctic fox,Vulpes,carni,Carnivora,unknown,12.5,-999.0,NaN,11.5,0.04450,3.380


## Standardizing missing values

Before we can deal with missing data, we must standardize the format of the missing values. That means

- converting strings like `"N/A"` or `"unknown"` to true NAs.
- converting code numbers like `-999` to true NAs.

By a "true NA", I mean **NumPy**'s `nan` value. (**pandas** also has a special value for missing data, `NA`, but it isn't yet widely supported.)

In [ ]:
from numpy import nan

To standardize the missing values, we replace them using the [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) method.

In [ ]:
msleep = msleep_dirty.replace(
    {"conservation": "unknown", "sleep_rem": -999},
    value=nan
)
msleep

,name,genus,vore,order,conservation,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,Cheetah,Acinonyx,carni,Carnivora,lc,12.1,NaN,NaN,11.9,NaN,50.000
1,Owl monkey,Aotus,omni,Primates,NaN,17.0,1.8,NaN,7.0,0.01550,0.480
2,Mountain beaver,Aplodontia,herbi,Rodentia,nt,14.4,2.4,NaN,9.6,NaN,1.350
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc,14.9,2.3,0.133333,9.1,0.00029,0.019
4,Cow,Bos,herbi,Artiodactyla,NaN,4.0,0.7,0.666667,20.0,0.42300,600.000
...,...,...,...,...,...,...,...,...,...,...,...
78,Tree shrew,Tupaia,omni,Scandentia,NaN,8.9,2.6,0.233333,15.1,0.00250,0.104
79,Bottle-nosed dolphin,Tursiops,carni,Cetacea,NaN,5.2,NaN,NaN,18.8,NaN,173.330
80,Genet,Genetta,carni,Carnivora,NaN,6.3,1.3,NaN,17.7,0.01750,2.000
81,Arctic fox,Vulpes,carni,Carnivora,NaN,12.5,NaN,NaN,11.5,0.04450,3.380


Notice that the missing values in the `conservation` and `sleep_rem` columns now display as `NaN` or `null`, depending on your Jupyter notebook editor.

## Quantifying missing values

To decide on the best technique for handling missing data, we need to know how much there is. We can find the proportion of missing data in each column by combining the [`.isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html) and [`.mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) methods.

A value of zero means no missing data, and a value of one means all values in the column are missing.

In [ ]:
msleep.isna().mean()

name            0.000000
genus           0.000000
vore            0.084337
order           0.000000
conservation    0.493976
sleep_total     0.000000
sleep_rem       0.265060
sleep_cycle     0.614458
awake           0.000000
brainwt         0.325301
bodywt          0.000000
dtype: float64

`vore` has a small amount of missing data, with 8% missing. `sleep_cycle` has the most missing data, with 61% missing.

## Dropping rows with missing values

The simplest solution to handling missing data is to get rid of any rows where there is a missing value using the [.dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method.

In [ ]:
msleep.dropna()

,name,genus,vore,order,conservation,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc,14.9,2.3,0.133333,9.1,0.00029,0.019
16,Lesser short-tailed shrew,Cryptotis,omni,Soricomorpha,lc,9.1,1.4,0.150000,14.9,0.00014,0.005
17,Long-nosed armadillo,Dasypus,carni,Cingulata,lc,17.4,3.1,0.383333,6.6,0.01080,3.500
19,North American Opossum,Didelphis,omni,Didelphimorphia,lc,18.0,4.9,0.333333,6.0,0.00630,1.700
21,Big brown bat,Eptesicus,insecti,Chiroptera,lc,19.7,3.9,0.116667,4.3,0.00030,0.023
24,European hedgehog,Erinaceus,omni,Erinaceomorpha,lc,10.1,3.5,0.283333,13.9,0.00350,0.770
39,Golden hamster,Mesocricetus,herbi,Rodentia,en,14.3,3.1,0.200000,9.7,0.00100,0.120
41,House mouse,Mus,herbi,Rodentia,nt,12.5,1.4,0.183333,11.5,0.00040,0.022
63,Laboratory rat,Rattus,herbi,Rodentia,lc,13.0,2.4,0.183333,11.0,0.00190,0.320
66,Eastern american mole,Scalopus,insecti,Soricomorpha,lc,8.4,2.1,0.166667,15.6,0.00120,0.075


In this case, we've droppped most of the dataset, going from 83 rows to 12. This is far from ideal.

Dropping data is only a suitable solution if there is only a very small amount of missing data.

## Separating the dataset by column data type

In **scikit-learn**, numeric data and unordered categorical data currently require different techniques for imputation. 

For ordered categorical data, you can either treat it in the same way as unordered categorical data, or transform in into integers with [`OrdinalEncoder()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html), and pretend that it is numeric. Here we'll take the former approach for simplicity.

The next step is to split the dataset into two by column type.

In [ ]:
msleep_num = msleep.select_dtypes("float")
msleep_num

,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,12.1,NaN,NaN,11.9,NaN,50.000
1,17.0,1.8,NaN,7.0,0.01550,0.480
2,14.4,2.4,NaN,9.6,NaN,1.350
3,14.9,2.3,0.133333,9.1,0.00029,0.019
4,4.0,0.7,0.666667,20.0,0.42300,600.000
...,...,...,...,...,...,...
78,8.9,2.6,0.233333,15.1,0.00250,0.104
79,5.2,NaN,NaN,18.8,NaN,173.330
80,6.3,1.3,NaN,17.7,0.01750,2.000
81,12.5,NaN,NaN,11.5,0.04450,3.380


In [ ]:
msleep_cat = msleep.select_dtypes(["object", "category"])
msleep_cat

,name,genus,vore,order,conservation
0,Cheetah,Acinonyx,carni,Carnivora,lc
1,Owl monkey,Aotus,omni,Primates,NaN
2,Mountain beaver,Aplodontia,herbi,Rodentia,nt
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc
4,Cow,Bos,herbi,Artiodactyla,NaN
...,...,...,...,...,...
78,Tree shrew,Tupaia,omni,Scandentia,NaN
79,Bottle-nosed dolphin,Tursiops,carni,Cetacea,NaN
80,Genet,Genetta,carni,Carnivora,NaN
81,Arctic fox,Vulpes,carni,Carnivora,NaN


## Replacing with means or medians

The simplest (and stupidest) imputation approach is to just to replace missing values with the mean or median of the column.

We have two options: **scikit-learn**'s [`SimpleImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) and **pandas**'s [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html).

We'll focus on `SimpleImputer()` since it's easiest to transition to the more advanced imputers later, but check our working with `.fillna()`.

First we import the **impute** submodule of **scikit learn**, and create a `SimpleImputer()` object.

In [ ]:
import sklearn.impute as si
simp = si.SimpleImputer()

Now we call the [`.fit_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer.fit_transform) method. *Fitting* means calculating the mean of the columns, and *transforming* means replacing the missing values with those means.

One annoyance is the **scikit-learn** only cares about **NumPy** arrays, so we have to manually convert it back to a data frame.

In [ ]:
pd.DataFrame(simp.fit_transform(msleep_num), columns=msleep_num.columns)

,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,12.1,1.87541,0.439583,11.9,0.281581,50.000
1,17.0,1.80000,0.439583,7.0,0.015500,0.480
2,14.4,2.40000,0.439583,9.6,0.281581,1.350
3,14.9,2.30000,0.133333,9.1,0.000290,0.019
4,4.0,0.70000,0.666667,20.0,0.423000,600.000
...,...,...,...,...,...,...
78,8.9,2.60000,0.233333,15.1,0.002500,0.104
79,5.2,1.87541,0.439583,18.8,0.281581,173.330
80,6.3,1.30000,0.439583,17.7,0.017500,2.000
81,12.5,1.87541,0.439583,11.5,0.044500,3.380


Before we get into the results, let's try this again with **pandas** to see how it works. We calculate the mean of each column (fit), and fill the missing values with those means (transform).

In [ ]:
column_means = msleep_num.mean()
msleep_num.fillna(column_means)

,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,12.1,1.87541,0.439583,11.9,0.281581,50.000
1,17.0,1.80000,0.439583,7.0,0.015500,0.480
2,14.4,2.40000,0.439583,9.6,0.281581,1.350
3,14.9,2.30000,0.133333,9.1,0.000290,0.019
4,4.0,0.70000,0.666667,20.0,0.423000,600.000
...,...,...,...,...,...,...
78,8.9,2.60000,0.233333,15.1,0.002500,0.104
79,5.2,1.87541,0.439583,18.8,0.281581,173.330
80,6.3,1.30000,0.439583,17.7,0.017500,2.000
81,12.5,1.87541,0.439583,11.5,0.044500,3.380


Notice that we have the same result in each case, so our code is correct!

Just like dropping data, imputing with the mean or median is only a suitable solution if

1. There is only a very small amount of missing data, and
2. anecdotally it performs better when you have lots of features.

## Replacing with the most frequent value

For categorical columns, there is no mean, so an alternative is to use the mode. That is, the most frequent value in the column.

In [ ]:
simp_mf = si.SimpleImputer(strategy="most_frequent")
pd.DataFrame(simp_mf.fit_transform(msleep_cat), columns=msleep_cat.columns)

,name,genus,vore,order,conservation
0,Cheetah,Acinonyx,carni,Carnivora,lc
1,Owl monkey,Aotus,omni,Primates,lc
2,Mountain beaver,Aplodontia,herbi,Rodentia,nt
3,Greater short-tailed shrew,Blarina,omni,Soricomorpha,lc
4,Cow,Bos,herbi,Artiodactyla,lc
...,...,...,...,...,...
78,Tree shrew,Tupaia,omni,Scandentia,lc
79,Bottle-nosed dolphin,Tursiops,carni,Cetacea,lc
80,Genet,Genetta,carni,Carnivora,lc
81,Arctic fox,Vulpes,carni,Carnivora,lc


Again, this isn't ideal, but there aren't really any good alternatives built-in to **scikit-learn**.

## Using iterative methods to find the best replacement

A more sophisticated option is to use [`IterativeImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html). This fits a predictive model (a Bayesian Ridge Regression) to each column, one at a time using the non missing values in the other columns. By repeating this process several times (iterating), more and more missing data gets filled in.

As with replacing using the mean, this only works with numeric columns.

`IterativeImputer()` should be your default starting point for most imputation. As of **scikit learn** `1.0.2`, it's considered  experimental, so you need to enable it before using it.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
iimp = si.IterativeImputer()
pd.DataFrame(iimp.fit_transform(msleep_num), columns=msleep_num.columns)

,sleep_total,sleep_rem,sleep_cycle,awake,brainwt,bodywt
0,12.1,2.246016,0.427049,11.9,0.136127,50.000
1,17.0,1.800000,0.252989,7.0,0.015500,0.480
2,14.4,2.400000,0.348557,9.6,0.086455,1.350
3,14.9,2.300000,0.133333,9.1,0.000290,0.019
4,4.0,0.700000,0.666667,20.0,0.423000,600.000
...,...,...,...,...,...,...
78,8.9,2.600000,0.233333,15.1,0.002500,0.104
79,5.2,0.788509,0.665730,18.8,0.267429,173.330
80,6.3,1.300000,0.413741,17.7,0.017500,2.000
81,12.5,2.310000,0.335082,11.5,0.044500,3.380


Notice that this time, the first three mising values in `sleep_cycle` have been replaced with different values.

## Where can I learn more?

- DataCamp's [Dealing with Missing Data in Python](https://app.datacamp.com/learn/courses/dealing-with-missing-data-in-python), [Cleaning Data in Python](https://app.datacamp.com/learn/courses/cleaning-data-in-python) and [Machine Learning with scikit-learn](https://app.datacamp.com/learn/courses/machine-learning-with-scikit-learn) courses.
- pandas [Working with Missing Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) tutorial.
- scikit-learn [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html) tutorial.